<a href="https://colab.research.google.com/github/AmarnadhReddy055/Multi-Model-Hate-Speech-Detection-In-Memes/blob/main/FeatureMixing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
import torch
import pandas as pd
import numpy as np

In [3]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc

In [4]:
train_data=pd.read_csv('/content/drive/MyDrive/train_image.csv')
dev_data=pd.read_csv('/content/drive/MyDrive/cross_validation_image.csv')

In [5]:
train_data.dropna(inplace=True)
train_data.reset_index(inplace=True)
train_data.head()

,index,id,img,label,text,features
0,0,1235,01235.png,1,when you're feeling horny asf but your habibi ...,"[0.71356636, 0.4687377, 0.7601375, 0.47391602,..."
1,1,1236,01236.png,0,meanwhile at the isis strip club,"[0.7574666, 0.49860162, 0.8951834, 0.45049658,..."
2,2,1243,01243.png,0,"when your human says ""who' s a good girl?"" and...","[0.8522473, 0.49101305, 0.8114812, 0.43492022,..."
3,3,1245,01245.png,1,and that was the last nativity play my son was...,"[0.81344944, 0.42663395, 0.7078128, 0.45056176..."
4,4,1247,01247.png,1,you can't be racist if there is no other race,"[0.75893325, 0.44350454, 0.7556687, 0.39917853..."


In [6]:
dev_data.head()

,id,img,label,text,cleaned_text,features
0,1268,01268.png,0,father and daughter bonding time is the best l...,father daughter bonding time best like share a...,"[0.8011039, 0.44203076, 0.8107121, 0.47249195,..."
1,1456,01456.png,1,they see them rollin..... they hating..,see rollin hating,"[0.78343225, 0.4830486, 0.9525863, 0.50286084,..."
2,1726,01726.png,1,a real man loads the dishwasher every night,real man load dishwasher every night,"[0.7510135, 0.46772155, 0.8101114, 0.41914812,..."
3,1742,01742.png,0,in just one hour from now i'll only have 4 hou...,one hour ill hour left work mere hour,"[0.8190234, 0.49838206, 0.8504915, 0.44808054,..."
4,1765,01765.png,1,"after a girl dies, what organ in her body stay...",girl dy organ body stay warm dick,"[0.85232306, 0.46574005, 0.7150654, 0.40895066..."


In [7]:
def covert_string_toarray_train(train_object):
  size_train=train_object.shape[0]
  print(size_train)
  for i in range(size_train):
    train_data['features'][i]=np.fromstring(train_data['features'][i][1:-1], sep=',')

In [8]:
def covert_string_toarray_dev(dev_object):
  size_dev=dev_object.shape[0]
  print(size_dev)
  for i in range(size_dev):
    dev_data['features'][i]=np.fromstring(dev_data['features'][i][1:-1], sep=',')

In [9]:
covert_string_toarray_train(train_data['features'])

8499


<ipython-input-7-8996c8233cd4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['features'][i]=np.fromstring(train_data['features'][i][1:-1], sep=',')


In [10]:
covert_string_toarray_dev(dev_data['features'])

500


<ipython-input-8-fb9ef8d8594c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dev_data['features'][i]=np.fromstring(dev_data['features'][i][1:-1], sep=',')


In [11]:
X_train_image=[]
for i in range(train_data.shape[0]):
  X_train_image.append(np.array(train_data['features'][i]))
X_train_image=np.array(X_train_image)

In [12]:
X_dev_image=[]
for i in range(dev_data.shape[0]):
  X_dev_image.append(np.array(dev_data['features'][i]))
X_dev_image=np.array(X_dev_image)

In [13]:
X_train_image.shape

(8499, 2048)

In [14]:
X_dev_image.shape

(500, 2048)

Text-Features

In [15]:
pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 102.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 2

In [16]:
from simpletransformers.language_representation import RepresentationModel

In [17]:
X_train_text=list(train_data['text'])

In [18]:
X_dev_text=list(dev_data['text'])

In [19]:
model=RepresentationModel(model_type='bert',
                          model_name='bert-base-uncased',
                          use_cuda=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTextRepresentation: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
sentence_embedding_train=model.encode_sentences(X_train_text,combine_strategy='mean')

In [21]:
sentence_embedding_dev=model.encode_sentences(X_dev_text,combine_strategy='mean')

In [22]:
sentence_embedding_train.shape

(8499, 768)

In [23]:
sentence_embedding_dev.shape

(500, 768)

In [24]:
y_train=train_data['label']

In [25]:
y_dev=dev_data['label']

Faeture Mixing

1.Concatenation

In [26]:
sentence_embedding_train.shape

(8499, 768)

In [27]:
X_train_image.shape

(8499, 2048)

In [28]:
early_train= np.concatenate((sentence_embedding_train,X_train_image),axis=1)

In [29]:
sentence_embedding_dev.shape

(500, 768)

In [30]:
X_dev_image.shape

(500, 2048)

In [31]:
early_dev=np.concatenate((sentence_embedding_dev,X_dev_image),axis=1)

In [32]:
from sklearn.metrics import accuracy_score

In [33]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(random_state=42,max_iter=1000)
lr_model.fit(early_train, y_train)
y_val_pred = lr_model.predict(early_dev)
print("Accuracy:", accuracy_score(y_dev, y_val_pred ))

Accuracy: 0.544


In [34]:
fpr, tpr, thresholds = roc_curve(y_dev,y_val_pred)
aucroc = auc(fpr, tpr)
print("AUCROC: ", aucroc)

AUCROC:  0.544


In [35]:
from xgboost import XGBClassifier
model2=XGBClassifier()
model2.fit(early_train,y_train)
y_val_pred = model2.predict(early_dev)
print("Accuracy:", accuracy_score(y_dev, y_val_pred ))

Accuracy: 0.524


In [36]:
fpr, tpr, thresholds = roc_curve(y_dev,y_val_pred)
aucroc = auc(fpr, tpr)
print("AUCROC: ", aucroc)

AUCROC:  0.5239999999999999


In [37]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from keras.regularizers import l1, l2
import tensorflow as tf

2.Early Fusion

In [38]:
import tensorflow as tf

# Define the inputs
image_features = tf.keras.Input(shape=(2048,))
text_features = tf.keras.Input(shape=(768,))

# Concatenate the image and text features
concatenated_features = tf.keras.layers.Concatenate()([image_features, text_features])

# Pass the concatenated features through a Dense layer
x = tf.keras.layers.Dense(512, activation='relu')(concatenated_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Define the model
model = tf.keras.Model(inputs=[image_features, text_features], outputs=output)

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Fit the model
history = model.fit(x=[X_train_image,sentence_embedding_train],
                    y=y_train,
                    validation_data=([X_dev_image,sentence_embedding_dev], y_dev),
                    epochs=100,
                    batch_size=32,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5, restore_best_weights=True)])

Epoch 1/100
266/266 [==============================] - 3s 4ms/step - loss: 0.6720 - accuracy: 0.6309 - val_loss: 0.7196 - val_accuracy: 0.5020
Epoch 2/100
266/266 [==============================] - 1s 4ms/step - loss: 0.6353 - accuracy: 0.6415 - val_loss: 0.7307 - val_accuracy: 0.5000
Epoch 3/100
266/266 [==============================] - 1s 4ms/step - loss: 0.6238 - accuracy: 0.6413 - val_loss: 0.7348 - val_accuracy: 0.5000
Epoch 4/100
266/266 [==============================] - 1s 4ms/step - loss: 0.6180 - accuracy: 0.6411 - val_loss: 0.7382 - val_accuracy: 0.5000
Epoch 5/100
266/266 [==============================] - 1s 5ms/step - loss: 0.6259 - accuracy: 0.6413 - val_loss: 0.7061 - val_accuracy: 0.5000
Epoch 6/100
266/266 [==============================] - 1s 5ms/step - loss: 0.6133 - accuracy: 0.6413 - val_loss: 0.8130 - val_accuracy: 0.5000
Epoch 7/100
266/266 [==============================] - 1s 5ms/step - loss: 0.6137 - accuracy: 0.6411 - val_loss: 0.6950 - val_accuracy: 0.5000

In [39]:
y_pred=model.predict(x=[X_dev_image,sentence_embedding_dev])

16/16 [==============================] - 0s 2ms/step


In [40]:
y_pred = np.where(y_pred> 0.5, 1, 0)

In [41]:
print(classification_report(y_dev,y_pred))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67       250
           1       0.00      0.00      0.00       250

    accuracy                           0.50       500
   macro avg       0.25      0.50      0.33       500
weighted avg       0.25      0.50      0.33       500



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
fpr, tpr, thresholds = roc_curve(y_dev,y_pred)
aucroc = auc(fpr, tpr)
print("AUCROC: ", aucroc)

AUCROC:  0.5


3.Late fusion

In [43]:
# Late Fusion Example
import tensorflow as tf

# Image model
image_features = tf.keras.Input(shape=(2048,))
x = tf.keras.layers.Dense(512, activation='relu')(image_features)
x = tf.keras.layers.Dropout(0.5)(x)
image_output = tf.keras.layers.Dense(256, activation='relu')(x)

# Text model
text_features = tf.keras.Input(shape=(768,))
x = tf.keras.layers.Dense(512, activation='relu')(text_features)
x = tf.keras.layers.Dropout(0.5)(x)
text_output = tf.keras.layers.Dense(256, activation='relu')(x)

# Concatenate the image and text outputs
concatenated_features = tf.keras.layers.Concatenate()([image_output, text_output])
output = tf.keras.layers.Dense(1, activation='sigmoid')(concatenated_features)

# Define the model
model = tf.keras.Model(inputs=[image_features, text_features], outputs=output)

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Fit the model
history = model.fit(x=[X_train_image,sentence_embedding_train],
                    y=y_train,
                    validation_data=([X_dev_image,sentence_embedding_dev], y_dev),
                    epochs=100,
                    batch_size=32,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5, restore_best_weights=True)])

Epoch 1/100
266/266 [==============================] - 3s 5ms/step - loss: 0.6273 - accuracy: 0.6651 - val_loss: 0.7745 - val_accuracy: 0.5380
Epoch 2/100
266/266 [==============================] - 1s 4ms/step - loss: 0.5800 - accuracy: 0.7013 - val_loss: 0.7191 - val_accuracy: 0.5200
Epoch 3/100
266/266 [==============================] - 1s 4ms/step - loss: 0.5669 - accuracy: 0.7056 - val_loss: 0.8178 - val_accuracy: 0.5100
Epoch 4/100
266/266 [==============================] - 2s 6ms/step - loss: 0.5593 - accuracy: 0.7129 - val_loss: 0.8063 - val_accuracy: 0.5220
Epoch 5/100
266/266 [==============================] - 2s 7ms/step - loss: 0.5489 - accuracy: 0.7171 - val_loss: 0.7509 - val_accuracy: 0.5460
Epoch 6/100
266/266 [==============================] - 2s 7ms/step - loss: 0.5433 - accuracy: 0.7176 - val_loss: 0.7829 - val_accuracy: 0.5220
Epoch 7/100
266/266 [==============================] - 2s 7ms/step - loss: 0.5402 - accuracy: 0.7270 - val_loss: 0.8503 - val_accuracy: 0.5120

In [44]:
y_pred=model.predict(x=[X_dev_image,sentence_embedding_dev])

16/16 [==============================] - 0s 2ms/step


In [45]:
y_pred = np.where(y_pred> 0.5, 1, 0)

In [46]:
print(classification_report(y_dev,y_pred))

              precision    recall  f1-score   support

           0       0.51      0.95      0.66       250
           1       0.64      0.09      0.16       250

    accuracy                           0.52       500
   macro avg       0.57      0.52      0.41       500
weighted avg       0.57      0.52      0.41       500



In [47]:
fpr, tpr, thresholds = roc_curve(y_dev,y_pred)
aucroc = auc(fpr, tpr)
print("AUCROC: ", aucroc)

AUCROC:  0.52


4.Multi-Modal Fusion(attenction)

In [48]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Dense, Dropout, Attention, concatenate
from tensorflow.keras.models import Model

# Load text and image features
text_features =sentence_embedding_train
image_features = X_train_image
text_val_features = sentence_embedding_dev
image_val_features = X_dev_image
# Define input layers
text_input = Input(shape=(768,), name='text_input')
image_input = Input(shape=(2048,), name='image_input')

# Apply attention mechanisms to text and image features
text_attention = Attention()([text_input, text_input])
image_attention = Attention()([image_input, image_input])

# Concatenate attention output for both text and image features
concatenated = concatenate([text_attention, image_attention])

# Add some dense layers on top of the concatenated features
x = Dense(512, activation='relu')(concatenated)
x = Dense(512, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

# Define model architecture
model = Model(inputs=[text_input, image_input], outputs=output)

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit model
history=model.fit({'text_input': text_features, 'image_input': image_features}, y_train, epochs=100, batch_size=32,
                  validation_data=({'text_input': text_val_features, 'image_input': image_val_features}, y_dev),
                  callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5, restore_best_weights=True)])

Epoch 1/100
266/266 [==============================] - 3s 7ms/step - loss: 0.6546 - accuracy: 0.6397 - val_loss: 0.7186 - val_accuracy: 0.5180
Epoch 2/100
266/266 [==============================] - 2s 6ms/step - loss: 0.5959 - accuracy: 0.6887 - val_loss: 0.7052 - val_accuracy: 0.5560
Epoch 3/100
266/266 [==============================] - 2s 6ms/step - loss: 0.5990 - accuracy: 0.6854 - val_loss: 0.8702 - val_accuracy: 0.5060
Epoch 4/100
266/266 [==============================] - 1s 5ms/step - loss: 0.5798 - accuracy: 0.6982 - val_loss: 0.8195 - val_accuracy: 0.5040
Epoch 5/100
266/266 [==============================] - 1s 4ms/step - loss: 0.5900 - accuracy: 0.6958 - val_loss: 0.8716 - val_accuracy: 0.5040
Epoch 6/100
266/266 [==============================] - 1s 4ms/step - loss: 0.5707 - accuracy: 0.7083 - val_loss: 0.7509 - val_accuracy: 0.5500
Epoch 7/100
266/266 [==============================] - 1s 4ms/step - loss: 0.5703 - accuracy: 0.7070 - val_loss: 0.7770 - val_accuracy: 0.5420

In [49]:
y_pred=model.predict({'text_input': text_val_features, 'image_input': image_val_features})

16/16 [==============================] - 0s 2ms/step


In [50]:
y_pred = np.where(y_pred> 0.5, 1, 0)

In [51]:
print(classification_report(y_dev,y_pred))

              precision    recall  f1-score   support

           0       0.54      0.85      0.66       250
           1       0.63      0.26      0.37       250

    accuracy                           0.56       500
   macro avg       0.58      0.56      0.51       500
weighted avg       0.58      0.56      0.51       500



In [52]:
fpr, tpr, thresholds = roc_curve(y_dev,y_pred)
aucroc = auc(fpr, tpr)
print("AUCROC: ", aucroc)

AUCROC:  0.5559999999999999
